### INTRODUCTION

In [1]:
import numpy as np
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

In [43]:
requests.get('https://cro.justice.cz/robots.txt')

<Response [200]>

In [2]:
print(requests.get("https://cro.justice.cz/verejnost/funkcionari?q=starosta%20nov%C3%BD%20dv%C5%AFr").text) # Using selenium - not complete source

<!DOCTYPE html><html lang="en"><head><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><link rel="shortcut icon" href="/favicon.ico"><title>Centrální registr oznámení</title><link href="/static/css/main.9e2cc9a1.css" rel="stylesheet"></head><body><div id="root"></div><script type="text/javascript" src="/static/js/main.f1e77434.js"></script></body></html>


In [6]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [45]:
# CRO testing - selenium
driver = webdriver.Chrome("C:/Program Files (x86)/Google/Chrome/Application/chromedriver")

In [46]:
# exactly the page by stating "mayor" and the name of the municipality - we will use these two keywords when creating a loop,
# i.e. "mayor" and the name of individual municipalities, which will be uploaded from the csv file

driver.get('https://cro.justice.cz/verejnost/funkcionari?q=starosta%20brno-malom%C4%9B%C5%99ice') 

In [47]:
driver.quit()

### INITIAL CLASSES + SOME CODES NEEDED IN FUTURE
Including identified problems and questions for the consultation on Wednesday.

In [3]:
class InputData:
    '''
    Class intended for the preparation of input data containing the list of Czehc municipalities.
    
    It contains methods for loading data and creating list of urls, 
    which will be then used in the Downloader manager "Mayors".
    '''
    def __init__(self, data):
        '''
        Creates the InputData object with self.data that store input data to be prepared.
        '''
        self.data = data
        
    def loadData(self):
        '''
        Loades input data with municipalities and extracts individual names of municipalities.
        '''
        input_data = pd.read_csv(self.data, sep = ",")
        names = input_data.iloc[:,1].values
        return names
    
    def createLinks(self):
        '''
        Creates a list of urls which will be then needed for the class Mayors to download data about individual mayors.
        '''
        links = []
        for n in self.loadData():
            if (len(n.split(" "))) == 2:
                n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1]
            elif (len(n.split(" "))) == 3:
                n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1] + "%20" + n.split(" ")[2]
            elif (len(n.split(" "))) == 4:      
                n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1] + "%20" + n.split(" ")[2] + "%20" + n.split(" ")[3]
            elif (len(n.split(" "))) == 5:
                n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1] + "%20" + n.split(" ")[2] + "%20" + n.split(" ")[3] + "%20" + n.split(" ")[4]
            elif (len(n.split(" "))) == 7:
                 n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1] + "%20" + n.split(" ")[2] + "%20" + n.split(" ")[3] + "%20" + n.split(" ")[4] + "%20" + n.split(" ")[5] + "%20" + n.split(" ")[6]
            else: 
                n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n
            links.append(n1)
            
        return links

In [49]:
## Testing our class using testing data sample which was created below (containing only 4 entries; final input data has
#  more than 6000 entries. Please see section "ALL INPUT DATA" for furher details. 4 links as output.
Instance_Data = InputData("Test.csv")
links = Instance_Data.createLinks()
links

['https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Libhošť',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Krhová',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Nový%20Dvůr',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Starý%20Jičín']

**Question no. 1:**

If condition created since respective links (after specifying keyword "mayor" and the name of individual municipalities) differ with respect to names of municipalities, i.e. number of words - see also section "ALL INPUT DATA" below for further details. **Is there any way how to write the if condition with respect to the issues mentioned above?**

In [50]:
driver = webdriver.Chrome("C:/Program Files (x86)/Google/Chrome/Application/chromedriver") # Insert to class Mayors?

In [51]:
# Not complete - just few initial lines of codes as we identified many problems. Please see related questions below.
class Mayors:
    
    def __init__(self, urls):
        
        self.urls = urls
    
    def getName(self):
        mayors = []
        
        for u in self.urls:
            driver.get(u)
            soup_cro = BeautifulSoup(driver.page_source, 'lxml')
            for s in soup_cro:
                people = soup_cro.findAll('td', {'class':'text'})
                for p in people:
                    data_test = p.text
                    mayors.append(data_test)
        return mayors
            
# will also contain a method to convert output to the pd.data frame          
            

In [52]:
## Testing our class using testing data sample which was created below (containing only 4 entries; final input data has
## more than 6000 entries. Please see section "ALL INPUT DATA" for furher details. 4 links as output.
## Basically we need all information listed with each entry, specifically: Name, Surname, Organisation (municipality),
## Function, and the Period druing which the function was hold. The attribute Title can be omitted.
Instance_Mayor = Mayors(links)
Instance_Mayor.getName()

['Josef',
 'Klos',
 'Obec Libhošť',
 'místostarosta / zástupce starosty',
 '05. 11. 2018 - ',
 'Jiří',
 'Petrovský',
 'Obec Libhošť',
 'starosta',
 '05. 11. 2018 - ',
 'Karla',
 'Chybová',
 'Obec Libhošť',
 'místostarosta / zástupce starosty',
 '04. 11. 2014 - 05. 11. 2018',
 'Helena',
 'Šimíčková',
 'Obec Libhošť',
 'starosta',
 '01. 02. 2011 - 05. 11. 2018',
 'Lukáš',
 'Macíček',
 'Krhová, Obec Krhová',
 'místostarosta / zástupce starosty, místostarosta / zástupce starosty',
 '05. 11. 2018 - , 14. 05. 2017 - 05. 11. 2018',
 'Kateřina',
 'Halaštová',
 'Krhová, Obec Krhová',
 'starosta, starosta',
 '05. 11. 2018 - , 14. 05. 2015 - 05. 11. 2018',
 'Irena',
 'Jeníková',
 'Obec Nový Dvůr',
 'místostarosta / zástupce starosty',
 '30. 10. 2018 - ',
 'Lubor',
 'Skořepa',
 'Obec Nový Dvůr',
 'místostarosta / zástupce starosty',
 '09. 11. 2014 - 30. 10. 2018',
 'Martin',
 'Macoun',
 'Obec Nový Dvůr, Obec Nový Dvůr',
 'starosta, starosta',
 '09. 11. 2014 - 30. 10. 2018, 30. 10. 2018 - ',
 'Miro

**Question no. 2:**

Please see related examples and the final summary below.

In [27]:
# If you put this link into the browser you obtain four entries including people in a function of "místostarosta" and etc.
# so the solution would be to put keyword "starosta" BUT - see another code below
driver.get('https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Libhošť')
soup_cro = BeautifulSoup(driver.page_source, 'lxml') 
person = soup_cro.find("span", text = "starosta").parent.parent
person.text

'JiříPetrovskýing.Obec Libhošťstarosta05. 11. 2018 - '

In [28]:
# Continue: sometimes you have there "starosta, starosta" - so it cannot be done using keyword "starosta"
# If using selenium functions find by a part of text - we would also obtain místostarosta which we do not want
driver.get('https://cro.justice.cz/verejnost/funkcionari?q=starosta%20nov%C3%BD%20dv%C5%AFr')
soup_cro = BeautifulSoup(driver.page_source, 'lxml') 
person = soup_cro.find("span", text = "starosta, starosta")
person.text # just for illustration - not tracking particular element

'starosta, starosta'

In [29]:
# Sometimes you have more than wanted entries, in this case 446 - putting municipality "Tísek"
# will also load all people called "František"
# can be then modified in the final dataframe BUT we have more than 6000 municipalities
# and saving all entries will slow down a process a lot.
driver.get('https://cro.justice.cz/verejnost/funkcionari?q=starosta%20t%C3%ADsek')


So we are now struggling with this issue, i.e. how to narrow the entries to include only mayors ("starosta") and not people holding other functions. We would be happy to discuss this issue. The same applies for the municipalities - see the example below.

In [30]:
# Listing also other municipalities except for Prague 1
driver.get("https://cro.justice.cz/verejnost/funkcionari?q=starosta%20praha%201")

In [31]:
# Sometime the name of a municipality is written in a wrong way, but we guess that there is no solution for this
# problem and this must be solved by hand.
driver.get("https://cro.justice.cz/verejnost/funkcionari?q=starosta%20%C5%BEelechovice%20nad%20d%C5%99evnic%C3%AD")
driver.get("https://cro.justice.cz/verejnost/funkcionari?q=starosta%20%C5%BEelechovice%20nad%20d%C5%99envic%C3%AD")

**Question no.3:**

The issue with the scroll button - but we would like to solve the problems listed above firstly. See the example below.
So we will focus on this later.

In [36]:
# Not able to identify entries listed at the second page
driver.get("https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Praha%201")
soup_cro = BeautifulSoup(driver.page_source, 'lxml')
person_ok = soup_cro.find("span", text = "Bíro")
person_ok.text

'Bíro'

In [38]:
# Not able to identify entries listed at the second page
person_nope = soup_cro.find("span", text = "Samec")
person_nope.text

AttributeError: 'NoneType' object has no attribute 'text'

### ALL INPUT DATA
The list of municipalities entering a for loop.
Finding characteristics + creating test data sample.

In [4]:
df_all = pd.read_csv("All_data.csv", sep = ",")
df_all.head(5)

,Kód zastupitelstva,Název
0,500011,Želechovice nad Dřevnicí
1,500020,Petrov nad Desnou
2,500046,Libhošť
3,500054,Praha 1
4,500062,Krhová


In [5]:
names_all = df_all.iloc[:,1].values
names_all

array(['Želechovice nad Dřevnicí', 'Petrov nad Desnou', 'Libhošť', ...,
       'Tichá', 'Tísek', 'Trojanovice'], dtype=object)

In [6]:
# Charcateristics of our municipalities - the link with differ with the count of word in the name of a municipality
sizes = []
for i in names_all:
    l = len(i.split())
    sizes.append(l)

In [7]:
max(sizes) # finding a maximum - 7

7

In [8]:
import collections # Finding frequencies
collections.Counter(sizes)


Counter({3: 360, 1: 5175, 2: 787, 4: 53, 5: 5, 7: 1})

In [9]:
df_all['No_words'] = sizes
df_all.head(5)

,Kód zastupitelstva,Název,No_words
0,500011,Želechovice nad Dřevnicí,3
1,500020,Petrov nad Desnou,3
2,500046,Libhošť,1
3,500054,Praha 1,2
4,500062,Krhová,1


In [40]:
# Creating test data sample
x = df_all.loc[df_all["No_words"] == 1].head(2)
y = df_all.loc[df_all["No_words"] == 2].tail(2)
test_1 = pd.concat([x, y], join = "outer")
test_1

,Kód zastupitelstva,Název,No_words
2,500046,Libhošť,1
4,500062,Krhová,1
6358,599654,Nový Dvůr,2
6374,599905,Starý Jičín,2


In [ ]:
test_1.to_csv("Test.csv", sep=',', encoding = 'utf-8', index = False) # which is then used above for classes

### SOME TESTING CODES
You can skip these - just for our purposes.

In [41]:
idx = np.linspace(1, 5, 5)
idx

array([1., 2., 3., 4., 5.])

In [204]:
test = []
for i in idx:
    x = df_all.loc[df_all["No_words"] == i].tail(2)
    test.append(x)
test = pd.concat(test)
test

,Kód zastupitelstva,Název,No_words
6379,599964,Tísek,1
6380,599999,Trojanovice,1
6358,599654,Nový Dvůr,2
6374,599905,Starý Jičín,2
6339,599468,Jeseník nad Odrou,3
6376,599930,Suchdol nad Odrou,3
6272,598739,Stará Ves nad Ondřejnicí,4
6300,599034,Zhoř u Mladé Vožice,4
3243,563455,Nová Ves u Mladé Vožice,5
3277,563790,Rychnov u Jablonce nad Nisou,5


In [208]:
names = test.iloc[:,1].values
names

array(['Tísek', 'Trojanovice', 'Nový Dvůr', 'Starý Jičín',
       'Jeseník nad Odrou', 'Suchdol nad Odrou',
       'Stará Ves nad Ondřejnicí', 'Zhoř u Mladé Vožice',
       'Nová Ves u Mladé Vožice', 'Rychnov u Jablonce nad Nisou'],
      dtype=object)

In [217]:
links

['https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Tísek',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Trojanovice',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Nový%20Dvůr',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Starý%20Jičín',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Jeseník%20nad%20Odrou',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Suchdol%20nad%20Odrou',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Stará%20Ves%20nad%20Ondřejnicí',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Zhoř%20u%20Mladé%20Vožice',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Nová%20Ves%20u%20Mladé%20Vožice',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Rychnov%20u%20Jablonce%20nad%20Nisou']

In [221]:
# Again simplified example to write a class:
test_1 = df_all.loc[df_all["No_words"] == 1].tail(2)
test_1
names = test_1.iloc[:,1].values
names

array(['Tísek', 'Trojanovice'], dtype=object)

In [224]:
urls = []
for n in names:
    n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n   
    urls.append(n1)
urls

['https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Tísek',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Trojanovice']

In [227]:
d3 = []
for u in urls:
    driver.get(u)
    soup_cro = BeautifulSoup(driver.page_source, 'lxml')
    d2 = soup_cro.find("title").text
    d3.append(d2)

In [228]:
d3

['Centrální registr oznámení', 'Centrální registr oznámení']

In [144]:
idx_1 = 
test_1 = []
for i in idx_1:
    x = df_all.loc[df_all["No_words"] == i].tail(2)
    test_1.append(x)
test_1 = pd.concat(test_1)

In [229]:
test_1

,Kód zastupitelstva,Název,No_words
6379,599964,Tísek,1
6380,599999,Trojanovice,1


In [196]:
names = test_1.iloc[:,1].values
names

array(['Tísek', 'Trojanovice', 'Nový Dvůr', 'Starý Jičín'], dtype=object)

In [200]:
d1 = []
for n in names:
    if (len(n.split(" "))) == 2:
        n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n.split(" ")[0] + "%20" + n.split(" ")[1]
    else:
        n1 = "https://cro.justice.cz/verejnost/funkcionari?q=starosta%20" + n   
    d1.append(n1)

In [201]:
d1

['https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Tísek',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Trojanovice',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Nový%20Dvůr',
 'https://cro.justice.cz/verejnost/funkcionari?q=starosta%20Starý%20Jičín']

In [160]:
test_2 = []
x = df_all.loc[df_all["No_words"] == 2].tail(2)
test_2.append(x)
test_2 = pd.concat(test_2)
test_2
names_2 = test_2.iloc[:,1].values
names_2

array(['Nový Dvůr', 'Starý Jičín'], dtype=object)

In [ ]:
driver.quit()